# **Deep Learning Method**
### Group 1 - Detection of Negation and Uncertainty

- Marino Oliveros Blanco NIU:1668563
- Pere Mayol Carbonell NIU:1669503
- Andreu Gascón Marzo NIU:1670919
- Judith Zaragoza NIU:1634071

## **Library and Data Loading**

In [23]:
# Libraries
import json
import re
from langdetect import detect # Library for language detection
from spellchecker import SpellChecker
import spacy
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, concatenate
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [18]:
# Loading the training json file
loading_train = open("negacio_train_v2024.json",)
training_data = json.load(loading_train)
print("The training set contains: ", len(training_data), " samples")

The training set contains:  254  samples


In [19]:
# Loading the json object
loading_test = open("negacio_test_v2024.json")
test_data = json.load(loading_test)
print("The test set contains: ", len(test_data), " samples")

The test set contains:  64  samples


## **Data Annotation**
- Set using ground truth word to either O, NEG, UNC, NSCO, USCO

In [20]:
def tag_words_from_json(json_data):
    # Extract text and predictions from JSON data
    text = json_data[0]['data']['text']
    predictions = json_data[0]['predictions'][0]['result']

    # Initialize counters
    counts = {'NEG': 0, 'NSCO': 0, 'UNC': 0, 'USCO': 0}

    # Initialize tokens and their coordinates
    tokens = []
    start_pos = 0

    # Split text into tokens while tracking their start and end positions
    for word in text.split():
        start = text.find(word, start_pos)
        end = start + len(word)
        tokens.append((word, start, end))
        start_pos = end

    # Initialize tags list
    tags = ['O'] * len(tokens)

    # Tag each word
    for pred in predictions:
        pred_start = pred['value']['start']
        pred_end = pred['value']['end']
        label = pred['value']['labels'][0]

        if label in counts:
            counts[label] += 1
            for i, (word, start, end) in enumerate(tokens):
                if start < pred_end and end > pred_start:
                    tags[i] = label

    # Combine tokens with tags
    tagged_text = [(token[0], (token[1], token[2]), tags[i]) for i, token in enumerate(tokens)]

    # Return tagged words with coordinates and counts
    return tagged_text, counts

In [25]:
# EXAMPLE USAGE
# Tag the words from the training data
tags, counts = tag_words_from_json(training_data)
print("Tagged Words:")
print(tags)
print("\nNumber of tokens in the entry:", len(tags))
print("\nCounts:")
print(counts)

Tagged Words:
[('nº', (1, 3), 'O'), ('historia', (4, 12), 'O'), ('clinica:', (13, 21), 'O'), ('**', (22, 24), 'O'), ('***', (25, 28), 'O'), ('***', (29, 32), 'O'), ('nºepisodi:', (33, 43), 'O'), ('********', (44, 52), 'O'), ('sexe:', (53, 58), 'O'), ('home', (59, 63), 'O'), ('data', (64, 68), 'O'), ('de', (69, 71), 'O'), ('naixement:', (72, 82), 'O'), ('16.05.1936', (83, 93), 'O'), ('edat:', (94, 99), 'O'), ('82', (100, 102), 'O'), ('anys', (103, 107), 'O'), ('procedencia', (108, 119), 'O'), ('cex', (120, 123), 'O'), ('mateix', (124, 130), 'O'), ('hosp', (131, 135), 'O'), ('servei', (136, 142), 'O'), ('urologia', (143, 151), 'O'), ('data', (152, 156), 'O'), ("d'ingres", (157, 165), 'O'), ('24.07.2018', (166, 176), 'O'), ('data', (177, 181), 'O'), ("d'alta", (182, 188), 'O'), ('25.07.2018', (189, 199), 'O'), ('08:54:04', (200, 208), 'O'), ('ates', (209, 213), 'O'), ('per', (214, 217), 'O'), ('***************,', (218, 234), 'O'), ('*****;', (235, 241), 'O'), ('****************,', (242, 2

In [33]:
def tag_words_from_json(json_data):
    # Initialize lists for all tagged texts and counts
    all_tagged_texts = []
    all_counts = []

    # Iterate through each entry in the JSON data
    for entry in json_data:
        # Extract text and predictions from JSON entry
        text = entry['data']['text']
        predictions = entry['predictions'][0]['result']

        # Initialize counters
        counts = {'NEG': 0, 'NSCO': 0, 'UNC': 0, 'USCO': 0}

        # Initialize tokens and their coordinates
        tokens = []
        start_pos = 0

        # Split text into tokens while tracking their start and end positions
        for word in text.split():
            start = text.find(word, start_pos)
            end = start + len(word)
            tokens.append((word, start, end))
            start_pos = end

        # Initialize tags list
        tags = ['O'] * len(tokens)

        # Tag each word
        for pred in predictions:
            pred_start = pred['value']['start']
            pred_end = pred['value']['end']
            label = pred['value']['labels'][0]

            if label in counts:
                counts[label] += 1
                for i, (word, start, end) in enumerate(tokens):
                    if start < pred_end and end > pred_start:
                        tags[i] = label

        # Combine tokens with tags
        tagged_text = [(token[0], (token[1], token[2]), tags[i]) for i, token in enumerate(tokens)]

        # Append the results to the lists
        all_tagged_texts.append(tagged_text)
        all_counts.append(counts)

    # Return the list of tagged words and counts for all entries
    return all_tagged_texts, all_counts

Number of tokens in the entry: 254
[('nº', (1, 3), 'O'), ('historia', (4, 12), 'O'), ('clinica:', (13, 21), 'O'), ('**', (22, 24), 'O'), ('***', (25, 28), 'O'), ('***', (29, 32), 'O'), ('nºepisodi:', (33, 43), 'O'), ('********', (44, 52), 'O'), ('sexe:', (53, 58), 'O'), ('home', (59, 63), 'O'), ('data', (64, 68), 'O'), ('de', (69, 71), 'O'), ('naixement:', (72, 82), 'O'), ('16.05.1936', (83, 93), 'O'), ('edat:', (94, 99), 'O'), ('82', (100, 102), 'O'), ('anys', (103, 107), 'O'), ('procedencia', (108, 119), 'O'), ('cex', (120, 123), 'O'), ('mateix', (124, 130), 'O'), ('hosp', (131, 135), 'O'), ('servei', (136, 142), 'O'), ('urologia', (143, 151), 'O'), ('data', (152, 156), 'O'), ("d'ingres", (157, 165), 'O'), ('24.07.2018', (166, 176), 'O'), ('data', (177, 181), 'O'), ("d'alta", (182, 188), 'O'), ('25.07.2018', (189, 199), 'O'), ('08:54:04', (200, 208), 'O'), ('ates', (209, 213), 'O'), ('per', (214, 217), 'O'), ('***************,', (218, 234), 'O'), ('*****;', (235, 241), 'O'), ('******

In [39]:
# Example usage with all entries
labels, counts_list = tag_words_from_json(training_data)

print("Number of entries in the entry:", len(labels))
print(labels[0]) # Print out an example entry
print(labels[0][0]) # Print out an example word
print(labels[0][0][2]) # Tag

Number of entries in the entry: 254
[('nº', (1, 3), 'O'), ('historia', (4, 12), 'O'), ('clinica:', (13, 21), 'O'), ('**', (22, 24), 'O'), ('***', (25, 28), 'O'), ('***', (29, 32), 'O'), ('nºepisodi:', (33, 43), 'O'), ('********', (44, 52), 'O'), ('sexe:', (53, 58), 'O'), ('home', (59, 63), 'O'), ('data', (64, 68), 'O'), ('de', (69, 71), 'O'), ('naixement:', (72, 82), 'O'), ('16.05.1936', (83, 93), 'O'), ('edat:', (94, 99), 'O'), ('82', (100, 102), 'O'), ('anys', (103, 107), 'O'), ('procedencia', (108, 119), 'O'), ('cex', (120, 123), 'O'), ('mateix', (124, 130), 'O'), ('hosp', (131, 135), 'O'), ('servei', (136, 142), 'O'), ('urologia', (143, 151), 'O'), ('data', (152, 156), 'O'), ("d'ingres", (157, 165), 'O'), ('24.07.2018', (166, 176), 'O'), ('data', (177, 181), 'O'), ("d'alta", (182, 188), 'O'), ('25.07.2018', (189, 199), 'O'), ('08:54:04', (200, 208), 'O'), ('ates', (209, 213), 'O'), ('per', (214, 217), 'O'), ('***************,', (218, 234), 'O'), ('*****;', (235, 241), 'O'), ('*****

## **Feature Extraction**

Relevant features to be extracted:
- POS Tags
- NER Tags
- Dependency Parsing trees
- Coordinates
- Negation Cues and Uncertainty Cues
- Context Window????

In [ ]:
""" PARA ADAPTAR 
#load spacy's spanish special.
nlp = spacy.load("es_core_news_md")
#to get the POS as a feature for each word.
def get_pos_tags(words):
    text = ' '.join(words) #convert the words to text.
    doc = nlp(text) #make it a doc using the function dowloaded before from spacy.

    #Create a dictionary where the key = word and the value = POS tag.
    pos_tags_dict = {token.text: token.pos_ for token in doc} 
    
    #print(f'Length of Words---> {len(pos_tags_dict)}') 
    return pos_tags_dict

pos_tags_all = []
for text in words:
    pos_tags_dict = get_pos_tags(words=text)
    pos_tags_all.append(pos_tags_dict)


pos_tags_feature = []
for texts in pos_tags_all:
    for word, tag in texts.items():
        pos_tags_feature.append([word,tag])
print(pos_tags_all[1])"""

In [ ]:
""" PARA ADAPTAR
words_feature = []
for i in range(len(words)):
    for word in words[i]:
        words_feature.append(word)"""

## **MODEL**

## **MODEL EVALUATION**